###分類任務

使用bert-base-chinese模型對新聞資料集做embeddings，接著訓練分類器。

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# 切換到位於 Google Drive 根目錄的 week11 資料夾
os.chdir('/content/drive/My Drive/week11')
os.listdir() # 確認目錄內容

Mounted at /content/drive


['udn_news.csv',
 'dict',
 'raw_data',
 'TaipeiSansTCBeta-Regular.ttf',
 'tmunlp_1.6B_WB_50dim_2020v1.bin.gz',
 'tmunlp_1.6B_WB_50dim_2020v1.bin']

In [ ]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer, models, util

In [ ]:
# 中文 bert-base-chinese
bert_ch = SentenceTransformer('google-bert/bert-base-chinese')

bert_ch.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

0

In [ ]:
udn = pd.read_csv("./udn_news.csv")
udn.head(3)

,system_id,artTitle,artDate,artCatagory,artUrl,artContent,dataSource
0,1,MLB／曾說「不是朋友」成真 大谷翔平解除追蹤水原一平,2024/3/21 9:56,運動,https://udn.com/news/story/11247/7845714,曾經是形影不離的戰友，大谷翔平與水原一平如今因金錢關係成為陌生人，大谷翔平的個人社群也解除對...,UDN
1,2,MLB／水原一平涉賭被解僱 日本網友震驚「巨大背叛」,2024/3/21 10:28,運動,https://udn.com/news/story/11247/7845808,日本棒球明星大谷翔平的翻譯水原一平傳出涉嫌竊取數百萬美元非法簽賭，已遭洛杉磯道奇解雇。大谷轉...,UDN
2,3,MLB／偷大谷翔平的錢還賭債！水原一平IG被球迷灌爆：你搞屁啊,2024/3/21 10:36,運動,https://udn.com/news/story/11247/7845856,美國職棒大聯盟（MLB）球星大谷翔平的隨行翻譯水原一平，因涉嫌竊取大谷至少450萬美元進行非...,UDN


In [ ]:
# 過濾 nan 的資料
udn = udn.dropna(subset=['artTitle'])
udn = udn.dropna(subset=['artContent'])
# 移除網址格式
udn["artContent"] = udn.artContent.apply(
    lambda x: re.sub("(http|https)://.*", "", x)
)
udn["artTitle"] = udn["artTitle"].apply(
    lambda x: re.sub("(http|https)://.*", "", x)
)
# 只留下中文字
udn["artContent"] = udn.artContent.apply(
    lambda x: re.sub("[^\u4e00-\u9fa5]+", "", x)
)
udn["artTitle"] = udn["artTitle"].apply(
    lambda x: re.sub("[^\u4e00-\u9fa5]+", "", x)
)

# 留下 content
udn["content"] = udn["artTitle"] + udn["artContent"]
udn = udn[["content", "artUrl", "artCatagory"]]  # 文章內容 文章連結
udn.head()

,content,artUrl,artCatagory
0,曾說不是朋友成真大谷翔平解除追蹤水原一平曾經是形影不離的戰友大谷翔平與水原一平如今因金錢關係...,https://udn.com/news/story/11247/7845714,運動
1,水原一平涉賭被解僱日本網友震驚巨大背叛日本棒球明星大谷翔平的翻譯水原一平傳出涉嫌竊取數百萬美...,https://udn.com/news/story/11247/7845808,運動
2,偷大谷翔平的錢還賭債水原一平被球迷灌爆你搞屁啊美國職棒大聯盟球星大谷翔平的隨行翻譯水原一平因...,https://udn.com/news/story/11247/7845856,運動
3,國內球評看水原一平醜聞形容比大谷結婚令人震撼道奇教士隊在南韓首爾進行新球季第場比賽前大谷翔平...,https://udn.com/news/story/11247/7845919,運動
4,大谷翔平恐受牽連水原一平稱他曾代為轉帳償還賭債後才改口美國職棒大聯盟明星大谷翔平的翻譯水原一...,https://udn.com/news/story/11247/7846177,運動


In [ ]:
udn["embeddings"] = udn.content.apply(lambda x: bert_ch.encode(x))
udn.head(3)

,content,artUrl,artCatagory,embeddings
0,曾說不是朋友成真大谷翔平解除追蹤水原一平曾經是形影不離的戰友大谷翔平與水原一平如今因金錢關係...,https://udn.com/news/story/11247/7845714,運動,"[0.61878633, -0.13968202, -0.13859954, 0.05170..."
1,水原一平涉賭被解僱日本網友震驚巨大背叛日本棒球明星大谷翔平的翻譯水原一平傳出涉嫌竊取數百萬美...,https://udn.com/news/story/11247/7845808,運動,"[0.38937178, -0.13959248, -0.23624946, 0.06701..."
2,偷大谷翔平的錢還賭債水原一平被球迷灌爆你搞屁啊美國職棒大聯盟球星大谷翔平的隨行翻譯水原一平因...,https://udn.com/news/story/11247/7845856,運動,"[0.60359246, -0.10200158, -0.2840907, 0.061311..."


In [ ]:
import numpy as np
from ast import literal_eval

In [ ]:

data = udn.copy()

X = data["embeddings"].apply(pd.Series)
y = data["artCatagory"]

# 把整個資料集七三切
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=777
)

print(X_train.head())
print(y_train.head())

           0         1         2         3         4         5         6    \
3232  0.444784  0.065882 -0.399327 -0.093501 -0.042365 -0.027646 -0.107268   
6408  0.410328 -0.086033 -0.296911  0.133511 -0.015810  0.012754 -0.272336   
5269  0.424380 -0.038221 -0.304500 -0.095474 -0.067336  0.195690  0.042882   
5811  0.336791 -0.071727 -0.468610  0.247080  0.082220  0.086640 -0.298778   
4781  0.332632  0.104276 -0.686178  0.081724  0.120209 -0.165410  0.023214   

           7         8         9    ...       758       759       760  \
3232 -0.133777 -0.230339 -0.139700  ...  0.106145 -0.210175  0.228320   
6408 -0.000721 -0.144832 -0.221995  ... -0.047226 -0.380887  0.208816   
5269  0.074572 -0.062906 -0.321693  ... -0.168546 -0.305021  0.186280   
5811  0.070805 -0.268437 -0.343612  ...  0.044431 -0.431023  0.255475   
4781 -0.007077 -0.229620 -0.211663  ...  0.340743  0.051793  0.123369   

           761       762       763       764       765       766       767  
3232 -0.026296 

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
print(y_pred[:10])

['兩岸' '運動' '兩岸' '全球' '股市' '股市' '兩岸' '股市' '兩岸' '全球']


In [ ]:

from sklearn.metrics import classification_report

In [ ]:
## Accuracy, Precision, Recall, F1-score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          全球       0.95      0.96      0.95       702
          兩岸       0.93      0.93      0.93       412
          股市       0.99      0.97      0.98       488
          運動       0.99      0.99      0.99       336

    accuracy                           0.96      1938
   macro avg       0.96      0.96      0.96      1938
weighted avg       0.96      0.96      0.96      1938



模型的準確率為 0.96，這表明模型在所有類別上表現相當不錯！